In [6]:
import numpy as np
import pandas as pd

In [2]:
def bernoulli(x, mu):
    # mu (size = (784,))
    # x (size = (784,))
    r = 1.0
    for x_i, mu_i in zip(x, mu):
        if x_i == 1:
            r *= mu_i
        else:
            r *= (1.0 - mu_i)
    return r

In [3]:
x = np.array([1,0,0,0,1,0,1,0,1,0] * 10)
y = np.random.rand(100)
y /= y.sum()

print(bernoulli(x,y))

2.003187097212377e-85


In [4]:
a = np.array([1,0,1])
b = np.array([0.3,0.5,0.6])


In [7]:
test = pd.Series([1,2,3])

In [11]:
print(test.shape)
print(test[0:].shape)

(3,)
(3,)


In [18]:
cls = []
cls.append(test.values)
print(cls)

[array([1, 2, 3], dtype=int64)]


In [17]:
print(test)

0    1
1    2
2    3
dtype: int64


In [19]:
np.argmax(test)


2